In [12]:
import pandas as pd
import numpy as np
import datetime as dt
from kiblib.utils.db import DbConn

In [13]:
db_conn = DbConn().create_engine()

In [14]:
# On importe le fichier rempli par Mathilde M. contenant les données du dernier semestre
action_culturelle = pd.read_excel("/home/kibini/FréquentationAC_2024_semestre2.xlsx",sheet_name="Action culturelle")
action_culturelle

/home/kibini/miniconda3/envs/kibini/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Evénement,Date,Porteur de projet,Animation,Lieu Action,Type d'action,Partenariat,public,Nombre de personnes venues,Notes,Jauge,Saturation de l'espace
0,NaN,2024-09-08,Hervé,Après-midi puzzles,Espace RDC,Jeux,NaN,Tout public,35.0,NaN,30.0,NaN
1,NaN,2024-09-10,Maïté,Atelier informatique pour débutants,Espace multimédia,Atelier,NaN,Adultes,9.0,NaN,10.0,NaN
2,NaN,2024-09-14,JB,Petit déjeuner jeux vidéo,La criée,Jeux,NaN,Familles,12.0,NaN,12.0,NaN
3,NaN,2024-09-14,Mathilde L.,Les Tulalus,Salle VDM,Club lecture,NaN,Adultes,NaN,NaN,15.0,NaN
4,NaN,2024-09-15,Adeline,"""Between"" : le Banquet des merveilles",La criée,Spectacle,Ballet du Nord,Tout public,45.0,NaN,50.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
89,NaN,2024-12-20,Céline,Radio à La Grand-Plage,Salle VDM,Atelier,NaN,Adultes,4.0,NaN,15.0,NaN
90,NaN,2024-12-21,Zina,Atelier de conversation en français,Salle VDM,Atelier,NaN,Adultes,7.0,NaN,10.0,NaN
91,NaN,2024-12-21,JB,Petit déjeuner jeux vidéo,La criée,Jeux,NaN,Familles,12.0,NaN,12.0,NaN
92,NaN,2024-12-21,Laetitia C.,Les racontées musicales,Salle d'heure du conte,Lecture/contes,NaN,Petite enfance,48.0,25 enfants et 23 adultes,40.0,NaN


In [15]:
action_culturelle.columns

Index(['Evénement', 'Date', 'Porteur de projet', 'Animation', 'Lieu Action',
       'Type d'action', 'Partenariat', 'public', 'Nombre de personnes venues',
       'Notes', 'Jauge', 'Saturation de l'espace'],
      dtype='object')

In [16]:
# On renomme les colonnes pour qu'elles correspondent aux colonnes dans stadb.action_culturelle
action_culturelle = action_culturelle.rename(columns={"Evénement":"evenement",
                                                      "Date":"date",
                                                      "Animation":"action",
                                                      "Lieu Action":"lieu",
                                                      "Type d'action":"type",
                                                      "Partenariat":"partenariat",
                                                      "public":"public",
                                                      "Nombre de personnes venues":"participants",
                                                      "Notes":"notes",
                                                      "Jauge":"jauge"})

In [17]:
action_culturelle.dtypes

evenement                         object
date                      datetime64[ns]
Porteur de projet                 object
action                            object
lieu                              object
type                              object
partenariat                       object
public                            object
participants                     float64
notes                             object
jauge                            float64
Saturation de l'espace           float64
dtype: object

In [18]:
action_culturelle['date'] = pd.to_datetime(action_culturelle['date']).dt.date

In [19]:
action_culturelle.dtypes

evenement                  object
date                       object
Porteur de projet          object
action                     object
lieu                       object
type                       object
partenariat                object
public                     object
participants              float64
notes                      object
jauge                     float64
Saturation de l'espace    float64
dtype: object

In [20]:
action_culturelle = action_culturelle.drop(labels=["Porteur de projet","Saturation de l'espace"],axis=1)

In [21]:
# On remet les colonnes dans le bon ordre
action_culturelle = action_culturelle[["date","action","lieu","type","public","partenariat","participants","jauge","evenement","notes"]]

action_culturelle

,date,action,lieu,type,public,partenariat,participants,jauge,evenement,notes
0,2024-09-08,Après-midi puzzles,Espace RDC,Jeux,Tout public,NaN,35.0,30.0,NaN,NaN
1,2024-09-10,Atelier informatique pour débutants,Espace multimédia,Atelier,Adultes,NaN,9.0,10.0,NaN,NaN
2,2024-09-14,Petit déjeuner jeux vidéo,La criée,Jeux,Familles,NaN,12.0,12.0,NaN,NaN
3,2024-09-14,Les Tulalus,Salle VDM,Club lecture,Adultes,NaN,NaN,15.0,NaN,NaN
4,2024-09-15,"""Between"" : le Banquet des merveilles",La criée,Spectacle,Tout public,Ballet du Nord,45.0,50.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
89,2024-12-20,Radio à La Grand-Plage,Salle VDM,Atelier,Adultes,NaN,4.0,15.0,NaN,NaN
90,2024-12-21,Atelier de conversation en français,Salle VDM,Atelier,Adultes,NaN,7.0,10.0,NaN,NaN
91,2024-12-21,Petit déjeuner jeux vidéo,La criée,Jeux,Familles,NaN,12.0,12.0,NaN,NaN
92,2024-12-21,Les racontées musicales,Salle d'heure du conte,Lecture/contes,Petite enfance,NaN,48.0,40.0,NaN,25 enfants et 23 adultes


# MODIFICATION DES DONNÉES DANS LA BASE

In [22]:
#action_culturelle.to_sql(name="stat_action_culturelle",con=db_conn,if_exists='append',index=False)

94